In [ ]:
#This is the notebook for tools_project

In [1]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [73]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat now ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat now ? Pizza
Where are you ? Boston,MA


'https://www.yelp.com/search?find_desc=Pizza&find_loc=Boston,MA'

In [74]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [75]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [76]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [77]:
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="WKxSOCh9LmgW4x4vtO2IFg" href="/adredir?ad_business_id=byf1dUfz_sqd2plTUB8tUQ&amp;campaign_id=RjztkfihebFegKUCkufGPA&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fgood-eats-pizza-and-subs-roxbury-crossing&amp;request_id=c7d9719172d53ed9&amp;signature=c32661327e31bc9c3c786e62d75ec607efdf6f38cf270665fae044eff150d40c&amp;slot=0"><span>Good Eats Pizza &amp; Subs</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="jVjZK3MWODpF2fsmAnswgw" href="/biz/regina-pizzeria-boston-28?osq=Pizza"><span>Regina Pizzeria</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="XrH8_WcbTVzqALJtOPAiPA" href="/biz/santarpios-pizza-boston-2?osq=Pizza"><span>Santarpio’s Pizza</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data

In [78]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this perticular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [ ]:
print(name,link)

In [79]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [20]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [ ]:
json_map

In [ ]:
print(latitude,longitude)

In [21]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [ ]:
category

In [22]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [ ]:
print(price_range)

In [23]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [ ]:
print(rating_value,review_count)

In [24]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [ ]:
print(phone_number)

In [25]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [ ]:
print(open_)

In [26]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

'66 Hereford StBoston, MA 02115'

In [27]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

'santouka.co.jp'

In [80]:
#want to put all infomations we want to get from one business in to one funtion
#input: url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
    dict_map = json.loads(json_map)
    latitude = dict_map["markers"][1]["location"]["latitude"]
    longitude = dict_map["markers"][1]["location"]["longitude"]

    category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    ratings = results_business_page.find("div", itemprop = 'aggregateRating')
    rating_value = ratings.find("meta").get('content')
    review_count = ratings.find("span").get_text()
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = None
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
        
    return(addr,latitude,longitude,category,price_range,rating_value,review_count,phone_number,open_,web)
    

In [ ]:
#test the function on cafe mogador
restaurant_url = "https://www.yelp.com/biz/cafe-mogador-brooklyn?osq=Restaurants"
get_info(restaurant_url)

In [81]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [ ]:
link_list

In [82]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [83]:
name_list

['Regina Pizzeria',
 'Santarpio’s Pizza',
 'Dirty Water Dough',
 'Galleria Umberto',
 'Ernesto’s',
 'Picco',
 'The Salty Pig',
 'Boston Kitchen Pizza',
 'Locale',
 'Figs by Todd English',
 'Penguin Pizza',
 'All Star Pizza Bar',
 'Dragon Pizza',
 'Regina Pizzeria',
 'Woody’s Grill and Tap',
 'Blaze Fast Fire’d Pizza',
 'Florina Pizzeria & Paninoteca',
 'Blaze Fast Fire’d Pizza',
 'Oath Pizza - South Station',
 'Ciao! Pizza & Pasta',
 'New York Pizza',
 'Pizza Stop',
 'Hot Box',
 'The Brewer’s Fork',
 'Italian Express Pizzeria',
 'Pastoral - Artisan Pizza, Kitchen & Bar',
 'Upper Crust Pizzeria',
 'Rina’s Pizzeria & Cafe',
 'Crush Pizza',
 'Stoked Wood Fired Pizza']

In [87]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [ ]:
#Create a dictionary showing the information of each restaurant
business_info={}

for i in range(len(name_list)):
    business_info[name_list[i]] = info_list[i]        

In [ ]:
business_info